In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.9 kB ? eta -:--:--
   --------------- ------------------------ 112.6/294.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 2.6 MB/s eta 0:00:00


In [4]:
df = pd.read_csv("./ToN_IoT_training_set.csv")

In [5]:
from sklearn.model_selection import train_test_split

In [7]:
x = df
y = df['type']
x, _x, y, _y = train_test_split(x, y, random_state=189, test_size=0.2, stratify=y)

In [11]:
x.to_csv('./training-set_0.csv', index=False)

In [12]:
_x.to_csv('./training-set_1.csv', index=False)

In [16]:
df['type'].value_counts()

type
normal        34955
backdoor      14102
dos           14094
password      14028
ransomware    13993
ddos          13981
injection     13975
xss           13974
scanning      13907
mitm            721
Name: count, dtype: int64

In [17]:
from ctgan import CTGAN

In [20]:
ctgan = CTGAN(verbose=True, epochs=10)

In [45]:
train_data = _x

In [49]:
ctgan_data = train_data[1:1]
for val in train_data['type'].unique():
    val_data = train_data[train_data['type'] == val].drop_duplicates()
    if len(val_data) > 50:
        val_data = val_data.sample(n=50, random_state=42)
    ctgan_data = pd.concat([ctgan_data, val_data])

In [48]:
ctgan.fit(ctgan_data, ctgan_data.columns)

Gen. (2.07) | Discrim. (-1.10): 100%|██████████| 10/10 [00:01<00:00,  9.81it/s]


In [84]:
data = train_data[1:1]

In [90]:
data = pd.concat([data, ctgan.sample(n=100, condition_column='type', condition_value='normal')]) 

In [91]:
data

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,192.168.1.33,41878,192.168.1.37,5357,tcp,http,0.128041,80,3653,S1,...,0,0,-,-,-,-,46,-,0,ransomware
1,192.168.1.31,91,151.101.30.49,6667,udp,ssl,0.047484,155,149,RSTR,...,0,0,-,-,-,-,-,-,0,mitm
2,192.168.1.79,56936,151.101.30.49,10502,udp,-,0.000054,3010,216,OTH,...,0,0,-,-,-,DNS_RR_unknown_type,-,-,1,normal
3,fe80::855:e4ef:de26:7aca,34296,216.58.203.109,443,tcp,dce_rpc,125.065890,37,53,REJ,...,0,0,-,-,-,-,46,-,1,xss
4,192.168.1.32,49535,196.216.169.11,4444,icmp,ssl,0.003205,260,1207,SH,...,0,0,-,-,-,-,-,-,1,scanning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,192.168.1.32,47092,52.139.250.253,1249,tcp,dns,0.080878,170,90,S2,...,0,0,-,-,-,-,-,-,1,backdoor
96,192.168.1.32,62946,216.58.199.34,2920,icmp,-,0.042546,235,463,OTH,...,0,0,-,-,-,-,-,-,0,xss
97,192.168.1.31,60834,196.216.169.11,993,tcp,-,60.760660,239,5884,S1,...,0,0,-,-,-,DNS_RR_unknown_type,-,F,0,ransomware
98,192.168.1.33,53988,192.168.1.79,10502,icmp,http,0.086189,2542,1342,S3,...,0,0,-,-,-,-,-,-,1,dos


In [94]:
from datetime import datetime

current_time = datetime.now()

formatted_current_time = current_time.strftime("%H:%M:%S %d/%m/%Y")
print("Current time:", formatted_current_time)



# Record the end time
end_time = datetime.now()

# Format the end time
formatted_end_time = end_time.strftime("%H:%M:%S")

# Print formatted end time
print("End time:", formatted_end_time)

# Calculate and print time taken
time_taken = end_time - current_time
print("Time taken: {:02d}:{:02d}:{:02d}".format(time_taken.seconds // 3600, (time_taken.seconds % 3600) // 60, time_taken.seconds % 60))

Current time: 14:26:40 16/04/2024
End time: 14:26:40
Time taken: 00:00:00
